# Integration tests for the GoFigr Python client

In [ ]:
TEST_RESULTS = []

In [ ]:
import json
from platform import platform
def sync():
    with open("integration_test.json", "w") as f:
        json.dump({
            'platform': platform(),
            'results': TEST_RESULTS
            }, f, indent=4)

# Load the extension and publish a figure from the same cell

In [ ]:
%load_ext gofigr
from gofigr.jupyter import *
import seaborn as sns
import matplotlib.pyplot as plt
import os

from uuid import uuid4
from gofigr.jupyter import _GF_EXTENSION as gfe

sns.set_style("darkgrid")
sns.set_context("talk") 

configure(workspace=None,
          auto_publish=True,
          analysis=FindByName("Integration tests", create=True),
          username=os.environ['GF_TEST_USER'],
          password=os.environ['GF_TEST_PASSWORD'],
          url="https://api-dev.gofigr.io")

gf = get_gofigr()

# Try plotting in the same cell as configure(). auto_publish will break, but we should at least
# get a warning
fig_title1 = str(uuid4())
plt.figure()
plt.scatter(range(10), range(10))
plt.title(fig_title1)

In [ ]:
from collections import defaultdict

def check_text(txt, min_len=10):
    return txt is not None and txt.strip() not in ["N/A", "NA"] and len(txt.strip()) >= min_len

def check_figure(test_name, workspace, analysis_name, fig_name, expected_revisions=1):
    res = defaultdict(lambda: True)
    res["test_name"] = test_name
    res["error"] = None
    
    try:    
        ana = workspace.get_analysis(analysis_name, create=False)
        fig = ana.get_figure(fig_name, create=False)

        res["number_of_revisions"] = len(fig.revisions) == expected_revisions

        for rev in fig.revisions:
            rev.fetch()
            res["notebook_name"] = res["notebook_name"] and \
                check_text(rev.metadata["notebook_name"])
            res["notebook_path"] = res["notebook_path"] and \
                check_text(rev.metadata["notebook_path"]) and os.path.exists(rev.metadata["notebook_path"])
            res["cell_id"] = res["cell_id"] and check_text(rev.metadata["cell_id"], min_len=5)
            
            for img in rev.image_data:
                image_key = f"image_{img.format}" + ("_watermark" if img.is_watermarked else "")
                res[image_key] = res[image_key] and len(img.data) > 1000

            for code in rev.code_data:
                txt = code.contents.strip()
                res["cell_code"] = res["cell_code"] and check_text(txt)
                
            for td in rev.text_data:
                txt = td.contents.strip()
                res['text'] = res["text"] and check_text(txt)
                
    except Exception as e:
        res["error"] = str(e)
            
    return dict(res.items())

TEST_RESULTS.append(check_figure("Configure and publish in one cell", gf.primary_workspace, "Integration tests", fig_title1))
sync()

# Manual call to publish

In [ ]:
iris = sns.load_dataset("iris")

title2 = str(uuid4())

plt.figure(figsize=(6, 6))
sns.scatterplot(data=iris, 
                x="sepal_length", 
                y="petal_length", 
                hue="species", 
                palette="tab10")
plt.title(title2)

publish(target=FindByName(title2, create=True),
        dataframes={"IRIS": iris})

In [ ]:
TEST_RESULTS.append(check_figure("Manual publish", gf.primary_workspace, "Integration tests", title2))
sync()

# Auto publish

In [ ]:
iris = sns.load_dataset("iris")

title3 = str(uuid4())
plt.figure(figsize=(6, 6))
sns.scatterplot(data=iris, 
                x="sepal_length", 
                y="petal_length", 
                hue="species", 
                palette="tab10")
plt.title(title3)

In [ ]:
TEST_RESULTS.append(check_figure("Auto publish", gf.primary_workspace, "Integration tests", title3))
sync()

# Add revision

In [ ]:
# Add a new revision to the same figure
plt.figure(figsize=(6, 6))
sns.scatterplot(data=iris, 
                x="sepal_length", 
                y="petal_length", 
                hue="species", 
                palette="tab10")
plt.title(title3)

In [ ]:
TEST_RESULTS.append(check_figure("Auto-publish second revision", gf.primary_workspace, "Integration tests", title3,
                                expected_revisions=2))
sync()

# Inline backend

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format="retina"

title4 = str(uuid4())
plt.figure()
plt.scatter(range(10), range(10))
plt.title(title4)

In [ ]:
TEST_RESULTS.append(check_figure("MPL inline backend", gf.primary_workspace, "Integration tests", title4))
sync()

# Mix-and-match manual and auto publish

In [ ]:
title5 = str(uuid4())
plt.figure(figsize=(6, 6))
sns.scatterplot(data=iris, 
                x="sepal_length", 
                y="petal_length", 
                hue="species", 
                palette="tab10")
plt.title(title5)
publish()

plt.figure(figsize=(4, 4))
sns.scatterplot(data=iris, 
                x="sepal_length", 
                y="petal_length", 
                hue="species", 
                palette="Set1")
plt.title(title5)

In [ ]:
TEST_RESULTS.append(check_figure("Mix manual and auto-publish", gf.primary_workspace, "Integration tests", title5, expected_revisions=2))
sync()

In [ ]:
import pandas as pd
pd.DataFrame(TEST_RESULTS)

In [ ]:
with open("integration_test.json.done", "w") as f:
    f.write("done\n")

In [ ]:
!pwd